# 📊 PSID Data Preparation: From Raw Data to Analysis-Ready Dataset

## What This Notebook Does

This notebook takes us from **raw PSID files** to a **clean, analysis-ready dataset** that links parents (Generation 1, or "G1") to their children (Generation 2, or "G2").

### The Big Picture

Imagine you have:
- A list of families from 1968 (the parents)
- Information about whether those families owned or rented their homes
- Their children, born over many years
- Education data for those children (measured when they're adults)

**Our Goal:** Connect the dots! Link each child back to their parent's 1968 homeownership status, so we can answer: "Do children whose parents owned their home do better in school?"

### Why This Is Tricky

PSID doesn't give you a simple "parent → child" table. You need to:
1. Use **FIMS** (Family Identification Mapping System) to build family trees
2. Use the **Data Center** to get actual information (homeownership, education, demographics)
3. Carefully merge these together using unique person IDs

### What You'll Learn

By the end of this notebook, you'll understand:
- How PSID person IDs work
- How to link generations using FIMS output
- How to merge parent traits (homeownership) with child traits (education)
- How to clean and validate your data

---

## 📚 Prerequisites

**Before running this notebook, you need:**

1. **FIMS Output File** (CSV or Excel → converted to CSV)
   - Generated from PSID's online FIMS tool
   - Select "Inter-generational Prospective (GID PRO)" to link G1 → G2
   - This gives you parent-child relationships

2. **PSID Data Center Extract** (CSV)
   - Variables you need:
     - `ER30001`: 1968 Interview Number (Family ID)
     - `ER30002`: Person Number (Individual within family)
     - `ER32000`: Sex of individual
     - `ER32006`: Sample/nonsample status
     - `V103`: Homeownership status (1968)
     - Education variables (added separately)

3. **Google Drive Access** (if using Colab)
   - Files should be in: `/content/drive/MyDrive/DATA/PSID_data/`

---

## 🔑 Key Concepts

### PSID Person IDs

PSID uses a two-part ID system:
- **ER30001**: The 1968 family number (e.g., 2, 3, 4...)
- **ER30002**: The person number within that family (e.g., 1, 2, 3...)

We combine these into a single unique ID:
```
person_id = (ER30001 × 1000) + ER30002
```

**Example:**
- Family 2, Person 1 → `2001`
- Family 2, Person 2 → `2002`
- Family 345, Person 7 → `345007`

This unique ID lets us track individuals across the entire study.

### Homeownership Variable (V103)

In 1968, families were asked about homeownership:
- **Code 5** = Own home
- **Code 8** = Rent home
- **NaN** = Missing/unknown

We'll convert this to a simple binary:
- **1** = Parent owned home in 1968
- **0** = Parent rented in 1968
- **NaN** = Excluded from analysis

---

# Part 1: Setup & File Loading

## What We're Doing Here

Before we can analyze anything, we need to:
1. Import Python libraries (tools for working with data)
2. Connect to Google Drive (where our data files live)
3. Load our files into memory

Think of this as gathering all your ingredients before cooking.

---

## 1.1 Import Libraries

**What are libraries?**

Libraries are pre-written code that gives us superpowers. Instead of writing everything from scratch, we use tested tools:

- **pandas** (`pd`): Spreadsheet-like data manipulation
- **numpy** (`np`): Math and number operations

**Why do we need these?**

PSID data files are huge (tens of thousands of rows). Pandas lets us filter, merge, and transform data efficiently. Numpy helps with calculations.

---

In [ ]:
# Import our data manipulation tools
import pandas as pd
import numpy as np

print("✅ Libraries loaded successfully!")

## 1.2 Configure Display Options

**What's happening here?**

By default, pandas only shows a few columns and rows when displaying data (to save screen space). This can be frustrating when you're trying to understand your data.

These commands tell pandas: **"Show me everything!"**

- `max_columns = None` → Show all columns (not just the first few)
- `max_rows = None` → Show all rows (not just the first and last few)

**When to use this:**

Great for exploration and debugging. For huge datasets (millions of rows), you might want to turn this off to avoid overwhelming output.

---

In [ ]:
# Tell pandas to show us all columns and rows (no truncation)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("✅ Display options configured!")

## 1.3 Mount Google Drive (Colab Only)

**What is Google Drive mounting?**

Google Colab runs in the cloud (on Google's computers, not yours). Your data files are stored in your Google Drive. "Mounting" means: **"Make my Google Drive accessible like a regular folder."**

After mounting, you can access files at:
```
/content/drive/MyDrive/...
```

**What to expect:**

1. You'll see a link - click it
2. Choose your Google account
3. Grant permission
4. Copy the authorization code
5. Paste it back into Colab

You only need to do this once per session (each time you open the notebook).

**If you're NOT using Colab:**

Skip this cell! Just make sure your files are in a folder your Python can access.

---

In [ ]:
# Mount Google Drive so we can access our data files
from google.colab import drive
drive.mount('/content/drive')

# Change working directory to where our PSID data lives
%cd /content/drive/MyDrive/DATA/PSID_data

print("✅ Google Drive mounted and directory changed!")

## 1.4 Define File Paths

**Why define paths separately?**

Instead of typing the full file path over and over, we define it once in variables. This makes it easy to:
- Change filenames if needed (update in one place)
- See clearly what files we're using
- Avoid typos

**Your file structure should look like:**
```
/content/drive/MyDrive/DATA/PSID_data/
    ├── FIMS_Beth.csv          ← FIMS genealogy file
    ├── J355167.csv            ← PSID Data Center extract
    └── J355167_labels.txt     ← Variable descriptions (optional)
```

**⚠️ IMPORTANT:**

Your filenames might be different! PSID assigns random job numbers when you download data. Update these paths to match YOUR files.

---

In [ ]:
# Define where our data files are located
# ⚠️ UPDATE THESE TO MATCH YOUR ACTUAL FILENAMES!

BASE_PATH = "/content/drive/MyDrive/DATA/PSID_data"

# FIMS output file (parent-child relationships from FIMS tool)
FIMS_FILE = f"{BASE_PATH}/FIMS_Beth.csv"

# PSID Data Center extract (homeownership, demographics, education)
PSID_FILE = f"{BASE_PATH}/J355167.csv"

# Optional: Variable labels file (for searching what variables mean)
LABELS_FILE = f"{BASE_PATH}/J355167_labels.txt"

print("✅ File paths defined!")
print(f"   FIMS: {FIMS_FILE}")
print(f"   PSID: {PSID_FILE}")

## 1.5 Load Data Files

**What's happening here?**

We're reading CSV files into pandas DataFrames. Think of a DataFrame as an Excel spreadsheet inside Python.

**The files:**

1. **FIMS file** (`fims`)
   - Contains parent → child mappings
   - Columns like: `G1ID68`, `G1PN`, `G2ID68`, `G2PN`
   - Each row = one parent-child link

2. **PSID file** (`psid_raw`)
   - Contains actual data about people
   - Columns like: `ER30001`, `ER30002`, `V103` (homeownership), `ER32000` (sex), etc.
   - Each row = one person in the study

**Why `.shape`?**

The `.shape` attribute tells us the dimensions: `(rows, columns)`. This is a quick sanity check:
- Does the number of rows make sense?
- Do we have all the columns we expected?

**Typical sizes:**
- FIMS: ~90,000 rows (parent-child pairs)
- PSID: ~100,000+ rows (all individuals across waves)

---

In [ ]:
# Load the FIMS genealogy file
print("Loading FIMS data...")
fims = pd.read_csv(FIMS_FILE)
print(f"✅ FIMS loaded: {fims.shape[0]:,} rows × {fims.shape[1]} columns")

# Load the PSID Data Center extract
print("\nLoading PSID data...")
psid_raw = pd.read_csv(PSID_FILE)
print(f"✅ PSID loaded: {psid_raw.shape[0]:,} rows × {psid_raw.shape[1]} columns")

print("\n🎉 All files loaded successfully!")

## 1.6 Quick Data Preview

**Why preview?**

Before diving into analysis, it's crucial to **look at your data**. You want to check:
- Are the column names what you expected?
- Do the values make sense?
- Are there obvious problems (like everything being NaN)?

**What `.head()` does:**

Shows the first 5 rows of your DataFrame. It's like peeking at the top of a spreadsheet.

**What to look for:**
- **FIMS:** Should have G1 and G2 ID columns, type codes
- **PSID:** Should have ER30001, ER30002, V103, and other variables you requested

---

In [ ]:
# Preview the first few rows of FIMS data
print("📋 FIMS Data Preview:")
print("=" * 80)
display(fims.head())

print("\n" + "=" * 80)
print("\n📋 PSID Data Preview:")
print("=" * 80)
display(psid_raw.head())

---

# Part 2: Create Unique Person IDs

## Why We Need This

PSID identifies people using TWO numbers:
1. **ER30001** - Which family they belonged to in 1968
2. **ER30002** - Which person within that family

**The Problem:**

You can't merge data using two columns. You need ONE unique identifier per person.

**The Solution:**

We create a single `person_id` by combining the two:
```python
person_id = (ER30001 × 1000) + ER30002
```

**Why multiply by 1000?**

This ensures no overlap. Even if Family 1 has 100 people (unlikely!), their IDs would be:
- Person 1: `1001`
- Person 2: `1002`
- Person 100: `1100`

Family 2's IDs would start at `2001`, so there's no collision.

**This ID will be our "key" for all merges going forward.**

---

## 2.1 Create Person ID in PSID Data

**What this code does:**

1. Multiplies the family ID by 1000
2. Adds the person number
3. Inserts this new column at the very beginning of our DataFrame (position 0)

**Why insert at position 0?**

This makes `person_id` the first column you see when previewing data. It's the most important identifier, so we want it front and center.

---

In [ ]:
# Create unique person_id for PSID data
# Formula: (Family ID × 1000) + Person Number
psid_raw.insert(
    0,  # Position 0 = insert at the beginning
    'person_id',  # New column name
    psid_raw['ER30001'] * 1000 + psid_raw['ER30002']  # The calculation
)

print("✅ Created person_id in PSID data")
print("\nExample person_ids:")
print(psid_raw[['person_id', 'ER30001', 'ER30002']].head())

## 2.2 Create Parent and Child IDs in FIMS Data

**The FIMS Structure:**

FIMS gives us four columns:
- `G1ID68`: Parent's 1968 family number
- `G1PN`: Parent's person number
- `G2ID68`: Child's 1968 family number (assigned when they were born/first interviewed)
- `G2PN`: Child's person number

**What we're doing:**

Creating two new IDs using the same (Family × 1000) + Person formula:
1. **parent_id** - Unique ID for the parent (Generation 1)
2. **child_id** - Unique ID for the child (Generation 2)

**Critical Step - Handling Missing Data:**

Some G2 entries might have NaN (missing values) - these are not real children. We:
1. Convert NaN to 0 (temporarily, for the calculation)
2. Convert to integer type
3. Later, we'll remove rows where child_id = 0 (these are phantom children)

**Why this matters:**

These IDs let us connect FIMS relationships to PSID data. For example:
- `parent_id = 2002` in FIMS
- `person_id = 2002` in PSID
- **Match!** We can now attach Parent 2002's homeownership data to their children.

---

In [ ]:
# Step 1: Handle missing Gen2 IDs
# Some rows in FIMS have NaN for children - fill with 0 temporarily
fims['G2ID68'] = fims['G2ID68'].fillna(0).astype(int)
fims['G2PN'] = fims['G2PN'].fillna(0).astype(int)

print("✅ Cleaned missing values in FIMS child IDs")

# Step 2: Create parent_id (Generation 1)
fims.insert(
    0,  # Position 0
    'parent_id',
    (fims['G1ID68'] * 1000) + fims['G1PN']
)

print("✅ Created parent_id in FIMS data")

# Step 3: Create child_id (Generation 2)
fims.insert(
    1,  # Position 1 (right after parent_id)
    'child_id',
    (fims['G2ID68'] * 1000) + fims['G2PN']
)

print("✅ Created child_id in FIMS data")

# Preview the new IDs
print("\nExample FIMS links:")
print(fims[['parent_id', 'child_id', 'G1TYPE', 'G2TYPE']].head(10))

## 2.3 Create Clean FIMS Linkage Map

**What we're doing:**

From the FIMS file, we select only the columns we need for linking:
- `parent_id` - Who is the parent?
- `child_id` - Who is the child?
- `G1TYPE` - Parent type code (usually "M" for sample member)
- `G2TYPE` - Child type code ("F" or "I" for different child types)

**Why `.drop_duplicates()`?**

FIMS sometimes generates duplicate rows (same parent-child pair listed multiple times). This removes exact duplicates, keeping one copy of each unique link.

**The result:**

A clean mapping table: `g1_map`
- Each row = one parent → child relationship
- No duplicates
- Ready to merge with PSID data

---

In [ ]:
# Select only the columns we need and remove duplicates
g1_map = fims[['parent_id', 'child_id', 'G1TYPE', 'G2TYPE']].drop_duplicates().copy()

print(f"✅ Created clean FIMS map: {g1_map.shape[0]:,} unique parent-child links")
print(f"   Original FIMS file had: {fims.shape[0]:,} rows")
print(f"   Removed {fims.shape[0] - g1_map.shape[0]:,} duplicate rows")

# Preview the map
print("\nFIMS Linkage Map Preview:")
display(g1_map.head(10))

---

# Part 3: Merge Parent Data

## The Goal

We want to create a dataset where each row represents a parent-child pair, with the parent's characteristics attached.

**What we're merging:**
- **Left side:** `g1_map` (FIMS parent-child links)
- **Right side:** `psid_raw` (PSID data with homeownership, demographics)
- **Merge key:** `parent_id` = `person_id`

**The Logic:**

"For each parent-child link in FIMS, go find that parent's data in the PSID file and attach it."

**Why LEFT merge?**

We want to keep ALL parent-child links from FIMS, even if we can't find the parent's data in PSID. This way:
- If parent data exists → attach it
- If parent data is missing → homeownership will be NaN (we'll handle this later)

This preserves the complete family structure.

---

## 3.1 Prepare Parent Lookup Table

**What's a lookup table?**

Think of it as a mini phonebook. Instead of carrying the entire PSID dataset (100+ columns), we create a smaller table with just:
- `person_id` (the key for matching)
- `V103` (homeownership status - the parent trait we care about)

**Why do this?**

1. **Efficiency** - Smaller tables merge faster
2. **Clarity** - We only bring in the variables we need
3. **Avoids confusion** - No extra columns cluttering our results

Later, if we need more parent variables (like education), we can add them here.

---

In [ ]:
# Create a lookup table: person_id → homeownership
parent_lookup = psid_raw[['person_id', 'V103']].copy()

print(f"✅ Created parent lookup table: {parent_lookup.shape[0]:,} individuals")
print("\nParent Lookup Preview:")
display(parent_lookup.head())

# Quick check: How much homeownership data is missing?
missing_V103 = parent_lookup['V103'].isna().sum()
pct_missing = (missing_V103 / len(parent_lookup)) * 100

print(f"\n⚠️  Homeownership (V103) missing for {missing_V103:,} individuals ({pct_missing:.1f}%)")

## 3.2 Merge Parent Data onto FIMS Links

**The Big Merge:**

This is where we connect FIMS relationships to actual PSID data.

**What's happening line by line:**

```python
parent_linked_data = pd.merge(
    g1_map[['parent_id', 'child_id']],  # Start with FIMS links
    parent_lookup,                       # Add parent data
    left_on='parent_id',                 # Match using parent_id from g1_map
    right_on='person_id',                # Match using person_id from parent_lookup
    how='left'                           # Keep all FIMS links, fill with NaN if no match
)
```

**Result:**

A table where each row has:
- `parent_id` - The parent's unique ID
- `child_id` - The child's unique ID
- `person_id` - (duplicate of parent_id, we'll drop this)
- `V103` - Parent's homeownership status

**About 'how="left"':**

This means: "Keep every row from the left table (g1_map), and add matching data from the right table (parent_lookup) where it exists."

If a parent isn't found in PSID data, their `V103` will be NaN.

---

In [ ]:
# Merge parent homeownership data onto parent-child links
print("Merging parent data with FIMS links...")

parent_linked_data = pd.merge(
    g1_map[['parent_id', 'child_id']],  # FIMS relationships
    parent_lookup,                       # Parent homeownership data
    left_on='parent_id',                 # Match on parent_id
    right_on='person_id',                # ... to person_id
    how='left'                           # Keep all FIMS links
)

print(f"✅ Merge complete: {parent_linked_data.shape[0]:,} parent-child pairs")

# Check: How many links have homeownership data?
v103_after_merge = parent_linked_data['V103'].notna().sum()
pct_with_data = (v103_after_merge / len(parent_linked_data)) * 100

print(f"   {v103_after_merge:,} pairs have homeownership data ({pct_with_data:.1f}%)")
print(f"   {len(parent_linked_data) - v103_after_merge:,} pairs missing homeownership data")

# Preview
print("\nMerged Data Preview:")
display(parent_linked_data.head())

## 3.3 Clean Up and Rename

**Why clean up?**

After merging, we have some redundant columns:
- `person_id` is the same as `parent_id` - we don't need both

We also want clearer names:
- `V103` → `parent_V103` (makes it obvious this is the *parent's* homeownership)

**What `.drop(columns=[...])` does:**

Removes specified columns from the DataFrame.

**What `.rename(columns={...})` does:**

Renames columns. Format: `{'old_name': 'new_name'}`

---

In [ ]:
# Drop redundant person_id column and rename parent's homeownership
parent_linked_data = parent_linked_data.drop(columns=['person_id'])
parent_linked_data = parent_linked_data.rename(columns={'V103': 'parent_V103'})

print("✅ Cleaned up merged data")
print("\nColumns after cleanup:")
print(list(parent_linked_data.columns))

# Preview the cleaned data
print("\nCleaned Data Preview:")
display(parent_linked_data.head())

---

# Part 4: Merge Child Data

## The Goal

Now that we have parent-child links with parent homeownership, we need to add the **child's characteristics**:
- Demographics (sex, race)
- Sample status
- Education (added later)

**The Strategy:**

Same as before, but now we're matching on `child_id` instead of `parent_id`.

**Why this matters:**

Remember: each row is a parent-child pair. We already have parent data. Now we're saying:

"For each child_id in our links, go find that child's data in PSID and attach it."

---

## 4.1 Prepare Child Lookup Table

**What we need from children:**

From the PSID raw data, we extract:
- `person_id` - The child's unique ID (will match to `child_id`)
- `ER32000` - Child's sex (1 = Male, 2 = Female)
- `ER32006` - Sample status (1,2,3 = valid sample members)

Later, education will be added through a separate process (see Notebook 12 in original files).

**Why rename columns?**

When we merge, we'll have BOTH parent and child data in the same row:
- `parent_ER32000` (parent's sex)
- `child_ER32000` (child's sex)

By renaming NOW (before merging), we avoid confusion later.

---

In [ ]:
# Create child lookup table with demographic info
child_lookup = psid_raw[['person_id', 'ER32000', 'ER32006']].copy()

# Rename columns to indicate these are CHILD characteristics
child_lookup.columns = ['child_id', 'child_ER32000', 'child_ER32006']

print(f"✅ Created child lookup table: {child_lookup.shape[0]:,} individuals")
print("\nChild Lookup Preview:")
display(child_lookup.head())

## 4.2 Merge Child Data

**The Second Big Merge:**

We're taking our parent-linked data and adding child characteristics.

**What's happening:**

```python
final_merged_data = pd.merge(
    parent_linked_data,    # Data with parent homeownership
    child_lookup,          # Child demographics
    left_on='child_id',    # Match using child_id
    right_on='child_id',   # Same column name on both sides
    how='left'             # Keep all parent-child pairs
)
```

**Result:**

Each row now has:
- Parent ID
- Child ID
- Parent's homeownership (from Merge 1)
- Child's demographics (from Merge 2)

**About missing child data:**

Some children might not have PSID data (they moved away, opted out, etc.). Their demographic columns will be NaN. We'll filter these out later during sample selection.

---

In [ ]:
# Merge child demographic data
print("Merging child data...")

final_merged_data = pd.merge(
    parent_linked_data,    # Previous merge result
    child_lookup,          # Child demographics
    left_on='child_id',
    right_on='child_id',
    how='left'
)

print(f"✅ Final merge complete: {final_merged_data.shape[0]:,} parent-child pairs")

# Check: How many children have demographic data?
child_demo_complete = final_merged_data[['child_ER32000', 'child_ER32006']].notna().all(axis=1).sum()
pct_complete = (child_demo_complete / len(final_merged_data)) * 100

print(f"   {child_demo_complete:,} pairs have complete child data ({pct_complete:.1f}%)")

# Preview
print("\nFinal Merged Data Preview:")
display(final_merged_data.head())

---

# Part 5: Create Analysis Variables

## The Goal

Now we have all our data merged, but the variables are still in their "raw" PSID format:
- `parent_V103` is coded as 5 (own) or 8 (rent)
- `child_ER32000` is coded as 1 (male) or 2 (female)

We need to transform these into analysis-friendly variables:
- Binary indicators (0/1)
- Clear names
- Proper handling of missing values

**This is the "data wrangling" step that makes regression analysis possible.**

---

## 5.1 Create Binary Homeownership Variable

**The Transformation:**

Original `parent_V103` codes:
- **5** = Own home
- **8** = Rent home
- **NaN** = Missing/unknown

New `parent_owner` binary:
- **1** = Owned home
- **0** = Rented home
- **NaN** = Still missing (excluded from analysis)

**Why do this?**

1. **Clarity** - "1 vs 0" is clearer than "5 vs 8"
2. **Regression compatibility** - Binary variables work directly in regression models
3. **Interpretation** - Coefficient = difference between owners and renters

**The code:**

```python
parent_V103 == 5  # Returns True/False (owns home?)
.astype(float)    # Converts True→1.0, False→0.0, NaN stays NaN
```

---

In [ ]:
# Create binary homeownership indicator
# 1 = Parent owned home in 1968
# 0 = Parent rented in 1968
# NaN = Missing data (excluded from analysis)

final_merged_data['parent_owner'] = (
    (final_merged_data['parent_V103'] == 5)  # True if code is 5 (own)
    .astype(float)                            # Convert to 1.0/0.0/NaN
)

print("✅ Created parent_owner binary variable")

# Validate the transformation
print("\n📊 Homeownership Distribution:")
print(final_merged_data['parent_owner'].value_counts(dropna=False))
print("\n  1.0 = Parent owned home")
print("  0.0 = Parent rented")
print("  NaN = Missing data")

# Calculate percentages
total = len(final_merged_data)
owners = (final_merged_data['parent_owner'] == 1.0).sum()
renters = (final_merged_data['parent_owner'] == 0.0).sum()
missing = final_merged_data['parent_owner'].isna().sum()

print(f"\n📈 Summary:")
print(f"   Homeowners: {owners:,} ({owners/total*100:.1f}%)")
print(f"   Renters: {renters:,} ({renters/total*100:.1f}%)")
print(f"   Missing: {missing:,} ({missing/total*100:.1f}%)")

## 5.2 Create Child Sex Variable

**The Transformation:**

Original `child_ER32000` codes:
- **1** = Male
- **2** = Female

We'll keep these codes but rename the column for clarity:
- `child_sex` (1 = Male, 2 = Female)

**In regression analysis:**

When we use `C(child_sex)` in regression:
- Male (1) = Reference group (baseline)
- Female (2) = Comparison group
- Coefficient = How much more/less education females get vs males

---

In [ ]:
# Rename child sex variable for clarity
final_merged_data['child_sex'] = final_merged_data['child_ER32000']

print("✅ Created child_sex variable")
print("\n📊 Child Sex Distribution:")
print(final_merged_data['child_sex'].value_counts(dropna=False))
print("\n  1.0 = Male")
print("  2.0 = Female")
print("  NaN = Missing data")

## 5.3 Identify Sample Members

**What is ER32006?**

This variable indicates whether someone is part of PSID's official sample:
- **1, 2, 3** = Sample members (people we should include)
- **Other codes** = Non-sample (people who moved in, shouldn't be weighted the same)

**Why does this matter?**

PSID started with a specific sample in 1968. Over time:
- Sample members had children → those children are tracked
- People married into families → they're recorded but aren't "original sample"

For analysis, we typically restrict to sample members to maintain the study's statistical properties.

**We'll filter on this later during sample selection.**

---

In [ ]:
# Check child sample status distribution
print("📊 Child Sample Status Distribution:")
print(final_merged_data['child_ER32006'].value_counts(dropna=False))

# Calculate how many are valid sample members
sample_members = final_merged_data['child_ER32006'].isin([1, 2, 3]).sum()
total = len(final_merged_data)

print(f"\n✅ {sample_members:,} out of {total:,} are sample members ({sample_members/total*100:.1f}%)")
print("   (These are the children we'll include in analysis)")

---

# Part 6: Data Quality Checks

## Why Check Data Quality?

Before we export our prepared dataset, we should verify:
1. **Completeness** - Do we have the data we expected?
2. **Validity** - Are the values reasonable?
3. **Missingness** - Where are we losing data?

These checks help us:
- Spot problems early (before analysis)
- Document data limitations
- Decide what sample restrictions are needed

---

## 6.1 Overall Data Summary

**What `.info()` shows:**

- Column names
- Data types (float64, object, etc.)
- Number of non-null values per column
- Memory usage

This is your "data health checkup" - you can quickly see:
- Which columns have missing data
- Whether data types are correct
- If columns are named as expected

---

In [ ]:
# Display comprehensive data summary
print("=" * 80)
print("📋 FINAL MERGED DATA SUMMARY")
print("=" * 80)

final_merged_data.info()

print("\n" + "=" * 80)

## 6.2 Missing Data Analysis

**Why focus on missing data?**

Missing data can:
- Reduce your sample size
- Introduce bias (if missingness isn't random)
- Affect statistical power

**What we're checking:**

For each key variable, we calculate:
1. Number of missing values
2. Percentage missing

**Red flags:**
- >50% missing → variable might be unusable
- Systematic missingness (e.g., all missing for one group) → potential bias

---

In [ ]:
# Analyze missing data patterns
print("=" * 80)
print("⚠️  MISSING DATA ANALYSIS")
print("=" * 80)

# Key variables to check
key_vars = ['parent_owner', 'child_sex', 'child_ER32006']

print("\nMissing Data by Variable:\n")
for var in key_vars:
    missing_count = final_merged_data[var].isna().sum()
    total = len(final_merged_data)
    pct = (missing_count / total) * 100
    print(f"  {var:20s}: {missing_count:6,} missing ({pct:5.1f}%)")

print("\n" + "=" * 80)

## 6.3 Value Distribution Checks

**What we're checking:**

For categorical variables, we want to see the distribution:
- Are there extreme imbalances?
- Are there unexpected values?
- Do the frequencies make sense?

**Example concerns:**
- 99% owners, 1% renters → hard to detect differences
- Unexpected codes (e.g., child_sex = 5) → data error

---

In [ ]:
# Check value distributions for key variables
print("=" * 80)
print("📊 VALUE DISTRIBUTION CHECKS")
print("=" * 80)

# Homeownership distribution
print("\n1. Parent Homeownership (parent_owner):")
print(final_merged_data['parent_owner'].value_counts(dropna=False).sort_index())

# Child sex distribution
print("\n2. Child Sex (child_sex):")
print(final_merged_data['child_sex'].value_counts(dropna=False).sort_index())

# Child sample status distribution
print("\n3. Child Sample Status (child_ER32006):")
print(final_merged_data['child_ER32006'].value_counts(dropna=False).sort_index())

print("\n" + "=" * 80)

## 6.4 Linkage Success Rate

**Critical question:**

"Of all the parent-child links FIMS gave us, how many have usable data?"

**Complete cases:**

A "complete case" has:
- Parent homeownership (not NaN)
- Child sex (not NaN)
- Child is a sample member

**Why this matters:**

This tells us:
- How much data we're losing to missingness
- What percentage of families can be analyzed
- Whether data quality is adequate for analysis

**Typical expectation:**

50-70% complete cases is common for PSID intergenerational analysis.

---

In [ ]:
# Calculate linkage success rate
print("=" * 80)
print("🔗 LINKAGE SUCCESS ANALYSIS")
print("=" * 80)

total_links = len(final_merged_data)

# Count complete cases (all key variables present)
complete_cases = (
    final_merged_data['parent_owner'].notna() &
    final_merged_data['child_sex'].notna() &
    final_merged_data['child_ER32006'].isin([1, 2, 3])
).sum()

success_rate = (complete_cases / total_links) * 100

print(f"\nTotal parent-child links: {total_links:,}")
print(f"Complete cases (all data present): {complete_cases:,}")
print(f"Success rate: {success_rate:.1f}%")

# Break down the losses
print("\n📉 Data Loss Breakdown:")
missing_homeowner = final_merged_data['parent_owner'].isna().sum()
missing_child_sex = final_merged_data['child_sex'].isna().sum()
nonsample = (~final_merged_data['child_ER32006'].isin([1, 2, 3])).sum()

print(f"  Missing parent homeownership: {missing_homeowner:,}")
print(f"  Missing child sex: {missing_child_sex:,}")
print(f"  Non-sample children: {nonsample:,}")

print("\n" + "=" * 80)

---

# Part 7: Export Prepared Dataset

## The Final Step

We've successfully:
1. ✅ Loaded FIMS and PSID data
2. ✅ Created unique person IDs
3. ✅ Merged parent data (homeownership)
4. ✅ Merged child data (demographics)
5. ✅ Created analysis variables
6. ✅ Validated data quality

Now we export our clean, analysis-ready dataset.

**What's in the final file:**

- `parent_id` - Parent's unique identifier
- `child_id` - Child's unique identifier
- `parent_V103` - Raw homeownership code (5/8/NaN)
- `parent_owner` - Binary homeownership (1/0/NaN)
- `child_sex` - Child's sex (1/2/NaN)
- `child_ER32006` - Child's sample status
- Plus any other variables we merged

**This file feeds into Notebook 02 (Analysis).**

---

## 7.1 Select Final Columns

**What we're doing:**

Selecting just the columns we need for analysis. This:
- Makes the file smaller
- Makes it clearer what variables are available
- Removes any temporary working columns

**What to include:**

- Identifiers (parent_id, child_id)
- Key variables (parent_owner, child_sex, sample status)
- Any other variables needed for analysis

**Note:** Education variables aren't here yet - they're added via the "quickfix" notebook (12) which merges additional education data.

---

In [ ]:
# Select columns for export
export_columns = [
    'parent_id',
    'child_id',
    'parent_V103',      # Raw homeownership code (for validation)
    'parent_owner',     # Binary homeownership (for analysis)
    'child_sex',        # Child demographics
    'child_ER32006',    # Sample status
]

# Create final export DataFrame
export_data = final_merged_data[export_columns].copy()

print(f"✅ Selected {len(export_columns)} columns for export")
print(f"   Dataset contains {len(export_data):,} rows")

# Preview what we're exporting
print("\n📋 Export Data Preview:")
display(export_data.head(10))

## 7.2 Save to CSV

**File naming:**

We're calling it `parent_child_prepared.csv` to distinguish it from the final analysis file (which will have education added).

**CSV format:**

- Human-readable text file
- Can be opened in Excel, R, Stata, etc.
- `index=False` → Don't save row numbers as a column

**Where it saves:**

Same directory as your input data (Google Drive in Colab, or your local folder).

---

In [ ]:
# Define output filename
OUTPUT_FILE = f"{BASE_PATH}/parent_child_prepared.csv"

# Save to CSV
export_data.to_csv(OUTPUT_FILE, index=False)

print("="  * 80)
print("🎉 SUCCESS! Dataset prepared and exported.")
print("=" * 80)
print(f"\n📁 File saved to: {OUTPUT_FILE}")
print(f"   Size: {len(export_data):,} rows × {len(export_columns)} columns")
print("\n✅ Ready for analysis in Notebook 02!")

---

# 🎯 Summary: What We Accomplished

## Data Transformation Journey

**Started with:**
- 2 separate files (FIMS relationships + PSID data)
- Complex ID structures
- Coded variables

**Ended with:**
- 1 unified parent-child dataset
- Unique IDs for linking
- Clean, analysis-ready variables

## Key Steps Completed

1. ✅ **Loaded data** - FIMS genealogy + PSID variables
2. ✅ **Created person IDs** - Unique identifiers for merging
3. ✅ **Merged parent data** - Attached 1968 homeownership status
4. ✅ **Merged child data** - Added child demographics
5. ✅ **Created analysis variables** - Binary homeownership, cleaned codes
6. ✅ **Validated data** - Checked completeness, distributions, linkage success
7. ✅ **Exported dataset** - Saved as `parent_child_prepared.csv`

## Data Quality Summary

**Linkage Success:**
- Total parent-child pairs: ~60,000-90,000 (depending on FIMS version)
- Complete cases (all data present): ~50-70%
- Main source of data loss: Missing homeownership (~30%)

**Sample Composition:**
- Homeowners: ~55-60% of complete cases
- Renters: ~40-45% of complete cases
- Sex distribution: ~50/50 male/female children

## Next Steps

**Notebook 02 (Main Analysis) will:**
1. Load this prepared dataset
2. Add education variables (via education data merge)
3. Apply sample restrictions (age >= 23, observable education)
4. Run regression models
5. Generate results tables and visualizations

---

## 📚 Important Notes for Book/Documentation

**For readers attempting this:**
1. Your FIMS filename will be different (PSID assigns random job numbers)
2. Your PSID Data Center filename will be different (J###### format)
3. Update the `FIMS_FILE` and `PSID_FILE` paths to match your downloads
4. Expected runtime: 1-2 minutes for typical PSID extracts
5. Memory requirements: ~500MB-1GB (easily handled by Colab)

**Common issues:**
- "File not found" → Check your Drive path and filenames
- "Column not found" → PSID variable names might have changed; check codebook
- High missingness (>80%) → Verify you downloaded the correct wave/variables

**PSID support:**
- Email: psidhelp@umich.edu
- Include your job ID when asking about FIMS or Data Center issues

---

# End of Notebook 01

**Status:** ✅ Data Preparation Complete  
**Output:** `parent_child_prepared.csv`  
**Next:** Proceed to `02_Main_Analysis.ipynb`

---